# Integer minimum cost flows with separable convex cost objective

The Problem of finding the most likely mulitpath split for a payment pair on the lightning network can be reduced to an **integer minimum cost flows with separable convex cost objective** and yields a polynomial solver c.f.: https://twitter.com/renepickhardt/status/1385144337907044352

This notebook tries to implement a solution given in the Textbook Network Flows Network Flows: Theory, Algorithms, and Applications by Ravindra K. Ahuja, Thomas L. Magnanti and James B. Orlin

This approach follows mainly chapter 9, 10.2 and 14.5 of the Textbook. 

Other good resources are the Lecture series http://courses.csail.mit.edu/6.854/20/ by David Karger (http://people.csail.mit.edu/karger/) with lecture notes at http://courses.csail.mit.edu/6.854/current/Notes/ and more specificially http://courses.csail.mit.edu/6.854/current/Notes/n09-mincostflow.html and the relevant subset of the recorded videos at this playlist: https://www.youtube.com/playlist?list=PLaRKlIqjjguDXlnJWG2T7U52iHZl8Edrcv

Other good resources are the lecture nodes by Hochbaum: https://hochbaum.ieor.berkeley.edu/

I did not find an open source implementation. Code for similar problems seems to be part of this library: https://github.com/frangio68/Min-Cost-Flow-Class but it seems to only allow quadratic cost functions. 

Following previous research ( https://arxiv.org/abs/2103.08576 ) our cost function is f(x) = log(c) - log(c-x) which turns out to be convex (f'(x) = 1/(c-x) and f"(x) = 1/(c-x)^2 from which we can see that f"(x) > 0)

In [6]:
import json
import networkx as nx
import math

log = math.log2

In [99]:
def import_channel_graph():
    # retrieve this by: lightning-cli listchannels > listchannels.json
    f = open("listchannels.json")
    jsn = json.load(f)
    G = nx.Graph()
    for channel in jsn["channels"]:
        src = channel["source"]
        dest = channel["destination"]
        cap = int(int(channel["satoshis"])/1000)
        sid = channel["short_channel_id"]
        G.add_edge(src,dest,cap=cap,sid=sid)
    return G

channel_graph = import_channel_graph()

G = nx.DiGraph()
G.add_edge("S","A",cap=2)
G.add_edge("S","X",cap=1)
G.add_edge("A","B",cap=2)
G.add_edge("X","B",cap=9)
G.add_edge("X","Y",cap=7)
G.add_edge("Y","D",cap=4)
G.add_edge("B","D",cap=4)

def generate_null_flow(G):
    flow = {n:{} for n in G.nodes()}
    for u,v in G.edges():
        flow[u][v]=0
        flow[v][u]=0
    return flow

def next_hop(path):
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)
        
def augment_path(flow,path,amt):
    for src,dest in next_hop(path):
        flow[src][dest]+=amt

def cost(x,c):
    if c%2==0:
        #print(x)
        return 4* x**2
    else:
        return x**2
#    return log(c+1)-log(c+1-x)

SRC = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
DEST = "022c699df736064b51a33017abfc4d577d133f7124ac117d3d9f9633b6297a3b6a"
FLOW = 9200

In [116]:


def compute_delta_residual_network(G,x,delta,C=cost):
    residual = nx.DiGraph()
    for i, j in G.edges():
        #print(i,j)
        cap = G[i][j]["cap"]
        f = x[i][j]
        print("src: {} dest: {} cap: {} flow: {} delta: {}".format(i,j,cap,f,delta))
        if x[i][j] == 0 and x[j][i]==0: # no flow:             
            if delta<=cap:
                unit_cost = (C(f + delta, cap) - C(f,cap))/delta
                #print("add residual",i,j,unit_cost)
                
                residual.add_edge(i,j,weight=(C(f + delta, cap) - C(f,cap))/delta)
                residual.add_edge(j,i,weight=(C(f + delta, cap) - C(f,cap))/delta)
        if x[i][j] > 0 and x[j][i] > 0:
            print("FLOW IN BOTH DIRECTION CAN'T / SHOULDN'T HAPPEN",i,j, x[i][j],x[j][i])
        if x[i][j]>0:
            if f+delta <=cap:
                residual.add_edge(i,j,weight=(C(f + delta, cap) - C(f,cap))/delta)
            if delta <= f:#backward flow
                print("HERE!!!",C(f - delta, cap), - C(f,cap),delta)
                residual.add_edge(j,i,weight=(C(f - delta, cap) - C(f,cap))/(2*delta))
        else: # if flow was in other direction enter in the other direction
            f=x[j][i]
            if f+delta <=cap:
                residual.add_edge(j,i,weight=(C(f + delta, cap) - C(f,cap))/delta)
            if delta <= f:#backward flow
                residual.add_edge(i,j,weight=(C(f - delta, cap) - C(f,cap))/(2*delta))
        
        ## DEBUG
        if residual.has_edge(i,j):
            print(i,j,residual[i][j]["weight"])
        if residual.has_edge(j,i):
            print(j,i,residual[j][i]["weight"])
    return residual
    
#compute_delta_residual_network(channel_graph,x,10)

In [117]:
def apply_reduced_cost(R,pi):
    for i,j in R.edges():
        R[i][j]["rc"]=R[i][j]["weight"]+pi[i]-pi[j]
        ###print(i,j,R[i][j]["rc"])

In [127]:
def cpacity_scaling(s,d,U,G):
    x = generate_null_flow(G)
    pi = {n:0 for n in G.nodes}
    e = {n:0 for n in G.nodes}
    e[s]=U
    e[d]=-U
    delta = 2**int(log(U))
    while delta >= 1:
        print("{}-scaling phase".format(delta))
        R = compute_delta_residual_network(G,x,delta)
        
        to_remove=[]
        for i,j in R.edges():
            #test and correct reduced cost optimality condition
            if R[i][j]["weight"]+pi[i]-pi[j] <0:
                
                #might be too simplisic as flow in oposite direction might exist which we would have to decrease
                print("correct reduced cost condition: ",x[i][j],i,j,R[i][j]["weight"],pi[i],pi[j])
                #print("backwards: ",j,i,R[j][i]["weight"],pi[j],pi[i])
                if x[i][j]>=delta:
                    x[i][j]+=delta
                else:
                    x[j][i]+=delta
                to_remove.append((i,j))
                e[i]-=delta
                e[j]+=delta
                
        for i,j in to_remove:
            R.remove_edge(i,j)
        ###print(e)
        ###print(x) 
        #R = compute_delta_residual_network(G,x,delta)
        S = [n for n,k in e.items() if k>=delta]
        T = [n for n,k in e.items() if k<=-delta]
        print(S,T)
        while len(S) >0 and len(T)>0:
            k=S[-1]
            l=T[-1]
            S=S[:-1]
            T=T[:-1]
            #path = nx.dijkstra_path(R,k,l)
            apply_reduced_cost(R,pi)
            #distances, paths = nx.single_source_dijkstra(R,k,weight="rc")
            distances, paths = nx.single_source_bellman_ford(R,k,weight="rc")
            for n, d_value in distances.items():
                print(n,d_value)
                #if n in pi:
                pi[n]-=d_value
            path = paths[l]
            augment_path(x,path,delta)
            print("augmented path:", path, "with:", delta)
            e[k]-=delta
            e[l]+=delta
            ###print(e)
            print("flow: ",x)
        delta = int(delta/2)
    return x
#x = cpacity_scaling(SRC,DEST, FLOW, channel_graph)
x = cpacity_scaling("S","D", 2, G)


2-scaling phase
src: S dest: A cap: 2 flow: 0 delta: 2
S A 8.0
A S 8.0
src: S dest: X cap: 1 flow: 0 delta: 2
src: A dest: B cap: 2 flow: 0 delta: 2
A B 8.0
B A 8.0
src: X dest: B cap: 9 flow: 0 delta: 2
X B 2.0
B X 2.0
src: X dest: Y cap: 7 flow: 0 delta: 2
X Y 2.0
Y X 2.0
src: B dest: D cap: 4 flow: 0 delta: 2
B D 8.0
D B 8.0
src: Y dest: D cap: 4 flow: 0 delta: 2
Y D 8.0
D Y 8.0
['S'] ['D']
S 0
A 8.0
B 16.0
X 18.0
D 24.0
Y 20.0
augmented path: ['S', 'A', 'B', 'D'] with: 2
flow:  {'S': {'A': 2, 'X': 0}, 'A': {'S': 0, 'B': 2}, 'X': {'S': 0, 'B': 0, 'Y': 0}, 'B': {'A': 0, 'X': 0, 'D': 2}, 'Y': {'X': 0, 'D': 0}, 'D': {'B': 0, 'Y': 0}}
1-scaling phase
src: S dest: A cap: 2 flow: 2 delta: 1
HERE!!! 4 -16 1
A S -6.0
src: S dest: X cap: 1 flow: 0 delta: 1
S X 1.0
X S 1.0
src: A dest: B cap: 2 flow: 2 delta: 1
HERE!!! 4 -16 1
B A -6.0
src: X dest: B cap: 9 flow: 0 delta: 1
X B 1.0
B X 1.0
src: X dest: Y cap: 7 flow: 0 delta: 1
X Y 1.0
Y X 1.0
src: B dest: D cap: 4 flow: 2 delta: 1
HERE!!! 4 

In [123]:
x

{'A': {'B': 3, 'S': 0},
 'B': {'A': 0, 'D': 3, 'X': 2},
 'D': {'B': 0, 'Y': 0},
 'S': {'A': 3, 'X': 2},
 'X': {'B': 0, 'S': 0, 'Y': 3},
 'Y': {'D': 1, 'X': 0}}

In [47]:
#print("from {} to {}".format(SRC[:5],DEST[:5]))
for s,v in x.items():
    for d,amt in v.items():
        if amt > 0:
            print(s,d,amt, channel_graph[s][d]["cap"])

03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f 022c699df736064b51a33017abfc4d577d133f7124ac117d3d9f9633b6297a3b6a 1 15000
03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df 02247d9db0dfafea745ef8c9e161eb322f73ac3f8858d8730b6fd97254747ce76b 1025 16777
03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df 032679fec1213e5b0a23e066c019d7b991b95c6e4d28806b9ebd1362f9e32775cf 1024 9000
03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df 03cde60a6323f7122d5178255766e38114b4722ede08f7c9e0c5df9b912cc201d6 1 10000
032679fec1213e5b0a23e066c019d7b991b95c6e4d28806b9ebd1362f9e32775cf 022c699df736064b51a33017abfc4d577d133f7124ac117d3d9f9633b6297a3b6a 2049 16777
02247d9db0dfafea745ef8c9e161eb322f73ac3f8858d8730b6fd97254747ce76b 032679fec1213e5b0a23e066c019d7b991b95c6e4d28806b9ebd1362f9e32775cf 1025 16777
03cde60a6323f7122d5178255766e38114b4722ede08f7c9e0c5df9b912cc201d6 03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f 1 2